In [1]:
!pip install -q transformers torchaudio openai-whisper jiwer
!apt-get -y install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.8 MB/s

In [2]:
from google.colab import files
uploaded = files.upload()
video_path = list(uploaded.keys())[0]
print("✅ Uploaded:", video_path)

Saving video1705_01.mp4 to video1705_01.mp4
✅ Uploaded: video1705_01.mp4


In [3]:
# STEP 3: CHUYỂN VIDEO SANG .WAV (mono 16kHz)
audio_path = video_path.replace(".mp4", ".wav")
!ffmpeg -i "{video_path}" -ac 1 -ar 16000 "{audio_path}"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [4]:
#STEP 4: CHUẨN HÓA ÂM THANH
import torchaudio

waveform, sr = torchaudio.load(audio_path)
if sr != 16000:
    waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)

In [6]:
#STEP 5: WHISPER (OpenAI – Local chạy)
import whisper

def run_whisper(audio_path, model_size="medium"):
    model = whisper.load_model(model_size)
    result = model.transcribe(audio_path)
    return result["text"]

whisper_text = run_whisper(audio_path)
print("📄 Whisper (OpenAI):\n", whisper_text[:500])

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📄 Whisper (OpenAI):
  Xin kính chào quý vị và các bạn đang theo dõi chương trình trên kênh Toàn cảnh 24h. Chúng tôi luôn cập nhật vấn đề mới nhất, nóng nhất để gửi đến quý vị. Sau đây là phần tiên chi tiết. Ngày 16 tháng 5, Phòng Giáo dục và Đào tạo hiện Vũng Liêm, tỉnh Vĩnh Lâm Thông tin đã có báo cáo về việc xử lý sai phạm đối với bạp PTBH, hiệu trưởng một trường mẫu giáo trên địa bản. Trước đó, vào cuối tháng 12 năm 2024, Phòng Giáo dục và Đào tạo hiện Vũ Liêm nhận được hai hình ảnh trụp 3H, tay cầm trả bằng giấy


In [7]:
# STEP 6: WHISPER-LARGE (qua HuggingFace)
from transformers import AutoProcessor, WhisperForConditionalGeneration
import torch

def run_whisper_hf(audio_path, model_name="openai/whisper-large"):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    processor = AutoProcessor.from_pretrained(model_name)
    model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)

    wave, sr = torchaudio.load(audio_path)
    if sr != 16000:
        wave = torchaudio.transforms.Resample(sr, 16000)(wave)
    if wave.shape[0] > 1:
        wave = wave.mean(dim=0, keepdim=True)

    inputs = processor(wave.squeeze(0), sampling_rate=16000, return_tensors="pt").to(device)
    with torch.no_grad():
        ids = model.generate(inputs["input_features"])
    return processor.batch_decode(ids, skip_special_tokens=True)[0]

whisper_large_text = run_whisper_hf(audio_path)
print("📄 Whisper-Large (HF):\n", whisper_large_text[:500])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


📄 Whisper-Large (HF):
  Xin kính chào quý vị và các bạn đang theo dõi chương trình trên kênh Toàn Cảnh 24h Chúng tôi luôn cập nhật vấn đề mới nhất, nóng nhất để gửi đến quý vị. Sau đây là phần tin chi tiết.
